In [ ]:
# Essential imports (loads in <3s)
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import plotly.graph_objects as go

# Install ipywidgets if missing
try:
    import ipywidgets as widgets
except ImportError:
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets"])
    import ipywidgets as widgets

print("Setup complete")

In [ ]:
# Fast rule-based fraud scoring (no ML dependencies)
def calculate_risk_score(amount, hour, category, is_international):
    """
    Rule-based fraud scoring system
    """
    score = 0
    
    # High amount transactions
    if amount > 1000:
        score += 40
    
    # Unusual hours
    if hour < 6 or hour > 22:
        score += 30
        
    # International transactions
    if is_international:
        score += 25
        
    # High-risk categories
    high_risk_cats = ['electronics', 'travel', 'digital']
    if any(cat in str(category).lower() for cat in high_risk_cats):
        score += 15
        
    return min(100, score)

print("Rule-based scoring engine ready")

In [ ]:
# Build interactive transaction scoring dashboard
def create_scoring_dashboard():
    # Widget styling
    style = {'description_width': 'initial'}
    layout_full = widgets.Layout(width='95%')
    
    # Input controls
    w_amount = widgets.FloatSlider(value=150.0, min=0, max=5000, step=10,
                                  description='Amount (EUR):', style=style, layout=layout_full)
    w_hour = widgets.IntSlider(value=14, min=0, max=23,
                              description='Hour (0-23):', style=style, layout=layout_full)
    w_category = widgets.Dropdown(options=['retail', 'electronics', 'travel', 'food', 'digital'],
                                 value='retail', description='Merchant:', style=style, layout=layout_full)
    w_intl = widgets.Checkbox(value=False, description='International?', style=style)
    
    # Scan button
    btn_scan = widgets.Button(description='SCAN TRANSACTION', button_style='primary',
                             layout=widgets.Layout(width='100%', height='60px'))
    
    # Results area
    out_viz = widgets.Output()
    
    def on_scan(b):
        with out_viz:
            clear_output(wait=True)
            
            # Calculate risk
            score = calculate_risk_score(w_amount.value, w_hour.value, w_category.value, w_intl.value)
            
            # Decision logic
            if score < 20:
                status, color, msg = "APPROVED", "#2ecc71", "Low Risk: Transaction Authorized"
            elif score < 70:
                status, color, msg = "REVIEW", "#f39c12", "Medium Risk: SMS Verification Required"
            else:
                status, color, msg = "BLOCKED", "#c0392b", "CRITICAL RISK: Transaction Stopped"
            
            # Risk gauge
            fig = go.Figure(go.Indicator(
                mode="gauge+number", value=score,
                title={'text': f"<b>FRAUD RISK SCORE</b><br><span style='color:{color};font-size:1.5em'>{status}</span>"},
                gauge={'axis': {'range': [None, 100]},
                      'bar': {'color': color},
                      'steps': [{'range': [0, 20], 'color': "rgba(46, 204, 113, 0.3)"},
                               {'range': [20, 70], 'color': "rgba(243, 156, 18, 0.3)"},
                               {'range': [70, 100], 'color': "rgba(192, 57, 43, 0.3)"}],
                      'threshold': {'line': {'color': "black", 'width': 4}, 'value': score}}
            ))
            fig.update_layout(height=400, margin=dict(l=30, r=30, t=50, b=30))
            fig.show()
            
            print(f"\nResult: {msg}")
            print(f"Details: €{w_amount.value} | {w_hour.value}:00 | {w_category.value} | Intl={w_intl.value}")
    
    btn_scan.on_click(on_scan)
    
    # Layout
    left_panel = widgets.VBox([widgets.HTML("<h3>Transaction Input</h3>"),
                              w_amount, w_hour, w_category, w_intl, btn_scan],
                             layout=widgets.Layout(width='35%', padding='20px', border='1px solid #ddd'))
    dashboard = widgets.HBox([left_panel, widgets.VBox([out_viz], layout=widgets.Layout(width='65%'))])
    
    on_scan(None)
    return dashboard

print("Dashboard builder ready")

In [ ]:
# Launch interactive fraud detection system
print("Starting Model Deployment Simulation...")
dashboard = create_scoring_dashboard()
display(dashboard)
print("\nDashboard active. Adjust sliders and click SCAN TRANSACTION.")